# Operations

Examples using `xnd` with `gumath` integrated with other libraries

## Setup

First we need to import the some libraries

In [1]:
import json

from gumath import functions as fn

from importnb import Notebook

from IPython.display import display

import matplotlib as mpl
import matplotlib.pyplot as plt
import networkx as nx

from xnd import xnd

In [2]:
with Notebook(): 
    from utils import xnd_gumath

In [3]:
x = xnd({'a': [0, 1, 2], 'b': [4, 5, 6]})

In [4]:
x['a'] + x['b']

TypeError: unsupported operand type(s) for +: 'xnd' and 'xnd'

In [ ]:
from gumath import functions as fn

In [ ]:
x = xnd([[1, 2, 3], [4, 5, 6]], dtype='int64')
fn.add(x, x)

In [ ]:
x = xnd([[1, 2, 3], [4, 5, 6]], dtype='int64')
plt.plot(x)
plt.show()

In [ ]:
class xnd_gumath(xnd):
    def __new__(cls, *args, **kwargs):
        ops = {'add': 'add', 'sub': 'subtrac', 'mul': 'multiply'}
        
        map_types = {
            'int': 'int32',
            'float': 'float32'
        }
        
        def gen_fn(op):
            def _target(*args):
                l_args = list(args)
                for i, v in enumerate(args):
                    if not isinstance(v, xnd):
                        _type = type(v)
                        _type_name = (
                            _type.__name__ if _type.__name__ not in map_types else
                            map_types[_type.__name__]
                        )
                        l_args[i] = xnd(v, type=_type_name)
                
                if any([not v.type.isscalar() for v in l_args]):
                    for i, v in enumerate(l_args):
                        if v.type.isscalar():
                            _type = '1 * {}'.format(str(v.type))
                            l_args[i] = xnd_gumath([v.value], type=_type)
                    
                return getattr(fn, op)(*tuple(l_args))
            return _target
        
        for py_op, gumath_op in ops.items():
            setattr(cls, '__{}__'.format(py_op), gen_fn(gumath_op))
            
        for op in fn.__dir__():
            if op.startswith('__'):
                continue
            setattr(cls, op, gen_fn(op))
        
        return super().__new__(cls, *args, **kwargs)

x = xnd_gumath([1, 2, 3], type='3 * int32')

x * 5 + x.log()